# Spellchecker quest

Хтось наробив помилок у віршах Тараса Шевченка. Ваша задача -- виправити ці помилки і прочитати приховане повідомлення.

## Задача

Ви отримаєте тренувальні та тестувальні дані.

Тренувальні дані знаходяться в полі `lab.train_text`. Це звичайний нерозмічений текст. На ньому необхідно натренувати мовну модель. Підійде будь-яка. Я би радив feed-forward нейрону модель з токенізацією по літерах або BPE, бо це те, що ми проходили на останній лекції. Але n-грамна теж має спрацювати.

Тестувальні дані знаходиться в полі `lab.test_items`. Приклад одного елемента:

```json
{
  "text": "Співали б прозу, та по ножах,",
  "error_start": 23,
  "error_end": 28,
  "error": "ножах",
  "corrections": [
    "ногах",
    "йотах",
    "єнотах",
    "ножах",
    "нотах"
  ]
}
```

`error_start` та `error_end` вказують на місцезнаходження помилки в тексті (в символах). У данному прикладі, помилкою є `text[23:28]`, тобто слово "ножах".

`corrections` -- це список можливих виправлень.

Ваша задача -- натренувати мовну модель та за її допомогою обрати правильне виправлення серед запропонованих.


## Приховане повідомлення

Один приклад в `lab.test_items` дає можливість прочитати одну літеру прихованого повідомлення. Для цього знайдіть різницю між літерами слова з помилкою (`error`) та обраним моделлю виправленням. Надрукуйте цю літеру. Якщо слово з помилкою направді правильне, а таке теж буває, надрукуйте пробіл. Приклади:

```
Error               Correction     Надрукувати
-------------------------------------------
привіт               приліт        л
пні                  поні          о
баллет               балет         л
привіт               привіт        [пробіл]
```

Приховане повідомлення, яке ви побачите в результаті -- це рядок з віршу одного з українських авторів.

Відповідь на квест -- ім'я автора/ки у форматі "Ім'я Прізвище".

Полетіли! 🚀

In [1]:
!pip install --quiet --ignore-installed https://static.lp-nlp.com/pypi/lpnlp-2025.10.14-py3-none-any.whl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
import lpnlp

lab = lpnlp.start(
    email="denys.koval.shi.2022@lpnu.ua",                   # <----------- Заповніть це поле
    lab="quest_spellchecker"
    )

Удачі!


In [ ]:
# Якщо вам потрібні тренувальні дані у вигляді файлу на вашому комп'ютері,
# розкоментуйте цей код:

# with open("train.txt", "wt") as f:
#     f.write(lab.train_text)

# from google.colab import files
# files.download("train.txt")

## Мовна модель

Натренуйте свою мовну модель тут

In [3]:
print(lab.train_text[:330])

﻿ПРИЧИННА

Реве та стогне Дніпр широкий,
Сердитий вітер завива,
Додолу верби гне високі,
Горами хвилю підійма.
І блідий місяць на ту пору
Із хмари де-де виглядав,
Неначе човен в синім морі,
То виринав, то потопав.
Ще треті півні не співали,
Ніхто нігде не гомонів,
Сичі в гаю перекликались,
Та ясен раз у раз скрипів.


In [4]:
############################################

##### Ваш код для тренування моделі тут

import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import numpy as np

# Параметри
CONTEXT_SIZE = 5  # Розмір контексту (скільки літер використовувати для передбачення)
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001

# Створення словника
def build_vocab(text, min_freq=2):
    """Створює словник літер з тексту"""
    char_counts = Counter(text)
    vocab = {'<PAD>': 0, '<UNK>': 1}

    for char, count in char_counts.items():
        if count >= min_freq:
            vocab[char] = len(vocab)

    return vocab

vocab = build_vocab(lab.train_text, min_freq=1)
vocab_size = len(vocab)
print(f"Розмір словника: {vocab_size}")

# Створення навчальних даних
def create_training_data(text, vocab, context_size):
    """Створює пари (контекст, наступна літера) для навчання"""
    data = []

    # Перетворюємо текст в індекси
    text_indices = [vocab.get(char, vocab['<UNK>']) for char in text]

    # Створюємо послідовності
    for i in range(len(text_indices) - context_size):
        context = text_indices[i:i + context_size]
        target = text_indices[i + context_size]
        data.append((context, target))

    return data

print("Створення навчальних даних...")
training_data = create_training_data(lab.train_text, vocab, CONTEXT_SIZE)
print(f"Кількість навчальних прикладів: {len(training_data)}")

# Модель
class FeedForwardLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, context_size):
        super(FeedForwardLM, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, vocab_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # Flatten embeddings
        embeds = embeds.view(embeds.size(0), -1)
        out = self.relu(self.linear1(embeds))
        out = self.dropout(out)
        out = self.relu(self.linear2(out))
        out = self.dropout(out)
        out = self.linear3(out)
        return out

# Ініціалізація моделі
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Використовується пристрій: {device}")

model = FeedForwardLM(vocab_size, EMBEDDING_DIM, HIDDEN_DIM, CONTEXT_SIZE).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Тренування
print("\nПочинаємо тренування...")
model.train()

for epoch in range(EPOCHS):
    total_loss = 0

    # Перемішуємо дані
    np.random.shuffle(training_data)

    # Батчі
    for i in range(0, len(training_data), BATCH_SIZE):
        batch = training_data[i:i + BATCH_SIZE]

        contexts = torch.tensor([item[0] for item in batch], dtype=torch.long).to(device)
        targets = torch.tensor([item[1] for item in batch], dtype=torch.long).to(device)

        # Forward pass
        model.zero_grad()
        outputs = model(contexts)
        loss = loss_function(outputs, targets)

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / (len(training_data) / BATCH_SIZE)
    print(f"Епоха {epoch + 1}/{EPOCHS}, Loss: {avg_loss:.4f}")

print("Тренування завершено!")

############################################

Розмір словника: 118
Створення навчальних даних...
Кількість навчальних прикладів: 414263
Використовується пристрій: cuda

Починаємо тренування...
Епоха 1/10, Loss: 2.2350
Епоха 2/10, Loss: 2.0170
Епоха 3/10, Loss: 1.9542
Епоха 4/10, Loss: 1.9179
Епоха 5/10, Loss: 1.8934
Епоха 6/10, Loss: 1.8762
Епоха 7/10, Loss: 1.8623
Епоха 8/10, Loss: 1.8511
Епоха 9/10, Loss: 1.8409
Епоха 10/10, Loss: 1.8317
Тренування завершено!


# Читаємо між рядків

In [5]:
import math
import collections
from typing import List, Tuple


# Допоміжна фунція:
def get_letter(w1: str, w2: str) -> str:
    """Повертає літеру, якої відрізняються слова або пробіл для однакових слів.
    """

    letters1 = collections.Counter(w1)
    letters2 = collections.Counter(w2)

    diff = letters1 - letters2
    if len(diff) != 1:
        return " "

    return diff.most_common()[0][0]


def score_text(text: str, model, vocab) -> float:
    """Повертає log-probability для тексту (менше = краще)."""

    model.eval()
    device = next(model.parameters()).device

    # Перетворюємо текст в індекси
    text_indices = [vocab.get(char, vocab['<UNK>']) for char in text]

    if len(text_indices) <= CONTEXT_SIZE:
        return 0.0

    total_log_prob = 0.0
    count = 0

    with torch.no_grad():
        for i in range(len(text_indices) - CONTEXT_SIZE):
            context = text_indices[i:i + CONTEXT_SIZE]
            target = text_indices[i + CONTEXT_SIZE]

            context_tensor = torch.tensor([context], dtype=torch.long).to(device)
            output = model(context_tensor)

            # Обчислюємо log probability
            log_probs = torch.nn.functional.log_softmax(output, dim=1)
            total_log_prob += log_probs[0][target].item()
            count += 1

    # Повертаємо негативний log-probability (менше = краще)
    return -total_log_prob / count if count > 0 else float('inf')


# Можете змінювати параметри та весь цей код, якщо потрібно
def solve(model, vocab, test_items) -> Tuple[List[str], str]:
    """Повертає список виправлених слів для кожного з текстів в test_items та
    секретне повідомлення.
    """

    choices = []
    secret = []

    for item in test_items:
        scores = []
        for corr in item['corrections']:

            # Підставляємо слово-кандидат в текст
            text = item['text'][:item['error_start']] + corr + item['text'][item['error_end']:]

            # Рахуємо score тексту
            score = score_text(text, model, vocab)
            scores.append(score)

            # print(f'{score:.4f} {text}')

        # Сортуємо кандидатів на виправлення за score
        result = sorted(zip(scores, item['corrections']), key=lambda x: x[0])

        # Обираємо найкращу заміну
        best = result[0]
        best_word = best[1]
        choices.append(best_word)

        # Знаходимо чергову літеру повідомлення
        error = item['error']
        letter = get_letter(error, best_word)
        secret.append(letter)

    secret_message = ''.join(secret)

    return choices, secret_message

choices, secret_message = solve(model, vocab, lab.test_items)

lab.evaluate_accuracy(choices)
print("SECRET MESSAGE: ", secret_message)


Відповідь правильна ✅
accuracy = 0.73. Непогано! Спробуй розгадай приховане повідомлення
SECRET MESSAGE:   иНзн єш що тир люд на   ознаєш п о це чи ні у  і  а тВ  мїєд на мук  твоя  єдина  чі твої  од і більше  еб  не буд  з вт    а ційГземл  інші  од Ти  ть л ди іНші кохатимуть люди  добрі  аскав   мзлі


In [6]:
lab.answer("Василь Симоненко")   # Замініть на ім'я автора/ки розшифрованих рядків

Відповідь правильна ✅
Правильно! 🚀 Заповни тепер цю форму, будь ласка: https://tally.so/r/wkl0zZ


📝  Відправте посилання на цей colab або PDF з ним у відповідне завдання VNS
